In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.FIM import FIMModel

In [9]:
hparams = {
    'scale':'demo',
    'name':'fim',
    'batch_size':5,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'filter_num':150,
    'embedding_dim':300,
    'metrics':'auc,group_auc,mean_mrr,ndcg@5,ndcg@10',
    'device':'cuda:0',
    'attrs': ['title'],
    'epochs':10,
    'k':-1,
    'select':None,
    'save_step':0,
    'save_each_epoch':True,
    'train_embedding':False,
}
device = torch.device(hparams['device'])

In [3]:
vocab, loaders = prepare(hparams)

In [10]:
fimModel = FIMModel(vocab=vocab,hparams=hparams).to(device)

In [11]:
train(fimModel, hparams, loaders[0], loaders[1], loaders[2])

training...
epoch 1 , step 300 , loss: 1.5540: : 400it [00:10, 39.13it/s]
0it [00:00, ?it/s]saved model of epoch 1
testing...
3622it [00:16, 221.01it/s]
0it [00:00, ?it/s]evaluation results:{'group_auc': 0.5225, 'mean_mrr': 0.2243, 'ndcg@5': 0.2394, 'ndcg@10': 0.3034}
validating...
15062it [01:04, 232.18it/s]
evaluation results:{'group_auc': 0.6463, 'mean_mrr': 0.3222, 'ndcg@5': 0.3444, 'ndcg@10': 0.4021}
